In [3]:
import pandas as pd
import tensorflow_hub as hub
import openai 
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
import ast
import cx_Oracle
import re

from pathlib import Path
import sys
import json
import requests

# Read csv-files

In [4]:
prefix = Path.cwd().parent.parent
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
prefix

PosixPath('/home/skikk/Dokumente/uni/WS2324/MIT/llm-enterprise')

In [14]:
with open('../../config.json') as f:
  config = json.load(f)

In [4]:
df_q = pd.read_json((str(prefix) + '/data/50_queries.json'))

In [5]:
df_q

,question,sql
0,How many organisation units in human resources are relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);
1,How many organisation units in human resources are new and not relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);
2,COUNT the number of courses offered by each department.,"SELECT OFFER_DEPT_CODE, COUNT(*) AS course_COUNT FROM SUBJECT_ENROLLABLE GROUP BY OFFER_DEPT_CODE;"
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, academic_terms.term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM SUBJECT_SUMMARY, academic_terms WHERE SUBJECT_SUMMARY.term_code = academic_terms.term_code;"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt FROM WAREUSER.FCLT_BUILDING_ADDRESS_HIST GROUP BY fclt_building_key);
5,What is the average number of courses a student can enroll in?,"SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM mit_student_directory, SUBJECT_SUMMARY WHERE DEPARTMENT_CODE = department GROUP BY full_name);"
6,Which students are employed by mit?,"SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;"
7,What is the shortest course available this semester?,"SELECT DISTINCT subject_title, term_end_date - term_start_date FROM (SELECT MIN(term_end_date - term_start_date) AS min_duration FROM academic_terms), academic_terms, SUBJECT_SUMMARY WHERE SUBJECT_SUMMARY.term_code = ACADEMIC_TERMS.TERM_CODE AND term_end_date - term_start_date = min_duration;"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM wareuser.fac_building fb INNER JOIN wareuser.fac_building_address fba ON fb.FAC_BUILDING_KEY = fba.building_key;"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;


In [40]:
tables = []
for q in df_q['sql']:
    tables.append(Parser(q.upper().replace('WAREUSER.', '')).tables)
tables

[['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['SUBJECT_ENROLLABLE'],
 ['SUBJECT_SUMMARY', 'ACADEMIC_TERMS'],
 ['FCLT_BUILDING_ADDRESS_HIST'],
 ['MIT_STUDENT_DIRECTORY', 'SUBJECT_SUMMARY'],
 ['MIT_STUDENT_DIRECTORY', 'EMPLOYEE_DIRECTORY'],
 ['ACADEMIC_TERMS', 'SUBJECT_SUMMARY'],
 ['FAC_BUILDING', 'FAC_BUILDING_ADDRESS'],
 ['MIT_STUDENT_DIRECTORY'],
 ['LIBRARY_MATERIAL_STATUS', 'TIP_MATERIAL_STATUS'],
 ['ZIP_USA', 'ZIP_CANADA'],
 ['SUBJECT_SUMMARY'],
 ['FCLT_ROOMS', 'FCLT_BUILDING'],
 ['SPACE_DETAIL', 'FAC_BUILDING'],
 ['COURSE_CATALOG_SUBJECT_OFFERED', 'ACADEMIC_TERMS_ALL'],
 ['SUBJECT_SUMMARY'],
 ['EMPLOYEE_DIRECTORY', 'FAC_ROOMS', 'FAC_BUILDING_ADDRESS'],
 ['EMPLOYEE_DIRECTORY'],
 ['SUBJECT_SUMMARY'],
 ['MIT_STUDENT_DIRECTORY'],
 ['IAP_SUBJECT_CATEGORY'],
 ['ZIP_USA', 'ZIP_CANADA'],
 ['EMPLOYEE_DIRECTORY'],
 ['STUDENT_DEPARTMENT'],
 ['EMPLOYEE_DIRECTORY'],
 ['STUDENT_DEPARTMENT'],
 ['SUBJECT_SUMMARY'],
 ['TOP_LEVEL_DOMAIN'],
 ['SUBJECT_SUMMARY', 'ACADEMIC

In [10]:
from typing import *
def table(test:List[Tuple[str, str]]):
    print(test[0][1])


In [ ]:
"You are a OracleSQL expert. Given an input question, create a syntactically correct OracleSQL query to run and return ONLY the generated Query and nothing else.\
Pay close attention on which column is in which table. if context contains more than one tables then create a query by performing JOIN operation only using the column unitid for the tables.\
Follow these Instructions for creating syntactically correct SQL query:\
- Be sure not to query for columns that do not exist in the tables and use alias only where required.\
- Likewise, when asked about the average (AVG function) or ratio, ensure the appropriate aggregation function is used.\
- Pay close attention to the filtering criteria mentioned in the question and incorporate them using the WHERE clause in your SQL query.\
- If the question involves multiple conditions, use logical operators such as AND, OR to combine them effectively.\
- When dealing with date or timestamp columns, use appropriate date functions (e.g., DATE_PART, EXTRACT) for extracting specific parts of the date or performing date arithmetic.\
- If the question involves grouping of data (e.g., finding totals or averages for different categories), use the GROUP BY clause along with appropriate aggregate functions.\
- Consider using aliases for tables and columns to improve readability of the query, especially in case of complex joins or subqueries.\
- If necessary, use subqueries or common table expressions (CTEs) to break down the problem into smaller, more manageable parts."

## Identify the unique tables referred by the corpus 

In [3]:
test = {'A':'2345', 'B': '3454634546'}
for key in test:
    print(test[key])

2345
3454634546


In [41]:
unique_tables = set()
for  l in tables: 
    for t in l:
        unique_tables.add(t.upper())

In [46]:
unique_tables = sorted(list(unique_tables))
unique_tables

['ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'DRUPAL_EMPLOYEE_DIRECTORY',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FAC_ROOMS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_BUILDING_HIST',
 'FCLT_MAJOR_USE',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_COURSE_INSTRUCTOR',
 'LIBRARY_MATERIAL_STATUS',
 'LIBRARY_RESERVE_CATALOG',
 'LIBRARY_RESERVE_MATRL_DETAIL',
 'LIBRARY_SUBJECT_OFFERED',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_DETAIL',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA']

In [47]:
len(unique_tables)

39

In [11]:
df_q

,question,sql,tables
0,How many organisation units in human resources are relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources are new and not relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,COUNT the number of courses offered by each department.,"SELECT OFFER_DEPT_CODE, COUNT(*) AS course_COUNT FROM SUBJECT_ENROLLABLE GROUP BY OFFER_DEPT_CODE;",[SUBJECT_ENROLLABLE]
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, academic_terms.term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM SUBJECT_SUMMARY, academic_terms WHERE SUBJECT_SUMMARY.term_code = academic_terms.term_code;","[SUBJECT_SUMMARY, ACADEMIC_TERMS]"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt FROM WAREUSER.FCLT_BUILDING_ADDRESS_HIST GROUP BY fclt_building_key);,[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a student can enroll in?,"SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM mit_student_directory, SUBJECT_SUMMARY WHERE DEPARTMENT_CODE = department GROUP BY full_name);","[MIT_STUDENT_DIRECTORY, SUBJECT_SUMMARY]"
6,Which students are employed by mit?,"SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;","[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]"
7,What is the shortest course available this semester?,"SELECT DISTINCT subject_title, term_end_date - term_start_date FROM (SELECT MIN(term_end_date - term_start_date) AS min_duration FROM academic_terms), academic_terms, SUBJECT_SUMMARY WHERE SUBJECT_SUMMARY.term_code = ACADEMIC_TERMS.TERM_CODE AND term_end_date - term_start_date = min_duration;","[ACADEMIC_TERMS, SUBJECT_SUMMARY]"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM wareuser.fac_building fb INNER JOIN wareuser.fac_building_address fba ON fb.FAC_BUILDING_KEY = fba.building_key;","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,[MIT_STUDENT_DIRECTORY]


In [ ]:
#df_q = pd.concat([df_q, new_df], ignore_index=True)
df_q[['question', 'sql', 'tables', 'gpt_sql_nocontext','gpt_sql_global_schema','gpt_sql_log']]
#df_q

## Go thrugh the `unique_tables` list and save the respective schemas as proper CSV files 

In [48]:
for t in unique_tables: 
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    schema_file = str(prefix) + '/data/views/schema/' + t + '.csv'
    if os.path.isfile(schema_file): 
        s = pd.read_csv(schema_file, header=None)
        s = s.drop(1) # delete the "header" line 
        column_names = [h.strip() for h in s[0][0].split(';')]
        s_new = pd.DataFrame(list(s[0][1:].apply(lambda row: [c.strip() for c in row.split(';')])), columns=column_names)
        new_schema_file = str(prefix) + '/data/views/schema_new/' + t + '.csv'
        s_new.to_csv(new_schema_file, index=False)

## Map tables to queries 

In [49]:
queries = []
for t in sorted(list(unique_tables)):
    print(t)
    q = set()
    for i, T in enumerate(df_q['tables']): 
        if t in T: 
            q.add(i) 
    queries.append(list(q))

ACADEMIC_TERMS
ACADEMIC_TERMS_ALL
CIS_COURSE_CATALOG
COURSE_CATALOG_SUBJECT_OFFERED
DRUPAL_EMPLOYEE_DIRECTORY
EMPLOYEE_DIRECTORY
FAC_BUILDING
FAC_BUILDING_ADDRESS
FAC_ROOMS
FCLT_BUILDING
FCLT_BUILDING_ADDRESS_HIST
FCLT_BUILDING_HIST
FCLT_MAJOR_USE
FCLT_ROOMS
HR_ORG_UNIT
HR_ORG_UNIT_NEW
IAP_SUBJECT_CATEGORY
IAP_SUBJECT_DETAIL
IAP_SUBJECT_PERSON
IAP_SUBJECT_SESSION
LIBRARY_COURSE_INSTRUCTOR
LIBRARY_MATERIAL_STATUS
LIBRARY_RESERVE_CATALOG
LIBRARY_RESERVE_MATRL_DETAIL
LIBRARY_SUBJECT_OFFERED
MIT_STUDENT_DIRECTORY
SE_PERSON
SPACE_DETAIL
STUDENT_DEPARTMENT
SUBJECT_ENROLLABLE
SUBJECT_OFFERED
SUBJECT_SUMMARY
TIP_DETAIL
TIP_MATERIAL
TIP_MATERIAL_STATUS
TIP_SUBJECT_OFFERED
TOP_LEVEL_DOMAIN
ZIP_CANADA
ZIP_USA


In [50]:
queries

[[32, 34, 3, 7, 43, 15, 29],
 [32, 43, 15],
 [40],
 [34, 15],
 [42],
 [36, 37, 6, 39, 42, 17, 18, 23, 25],
 [8, 17, 14],
 [8, 17],
 [17],
 [4, 39, 45, 13, 46],
 [4],
 [39],
 [47],
 [39, 45, 13, 47, 46],
 [0, 1],
 [0, 1],
 [21],
 [32, 48],
 [32, 48],
 [32],
 [44],
 [10],
 [44],
 [44],
 [44],
 [35, 5, 6, 37, 9, 42, 45, 20],
 [36, 38],
 [49, 14],
 [24, 26],
 [2],
 [34, 41, 44, 15, 30, 31],
 [3, 5, 7, 43, 12, 16, 19, 27, 29],
 [31],
 [33, 10, 31],
 [10],
 [31],
 [28],
 [11, 22],
 [11, 22]]

In [51]:
df_t = pd.DataFrame(columns=['table', 'log'])
df_t['table'] = sorted(list(unique_tables))
df_t['log'] = queries
df_t

,table,log
0,ACADEMIC_TERMS,"[32, 34, 3, 7, 43, 15, 29]"
1,ACADEMIC_TERMS_ALL,"[32, 43, 15]"
2,CIS_COURSE_CATALOG,[40]
3,COURSE_CATALOG_SUBJECT_OFFERED,"[34, 15]"
4,DRUPAL_EMPLOYEE_DIRECTORY,[42]
5,EMPLOYEE_DIRECTORY,"[36, 37, 6, 39, 42, 17, 18, 23, 25]"
6,FAC_BUILDING,"[8, 17, 14]"
7,FAC_BUILDING_ADDRESS,"[8, 17]"
8,FAC_ROOMS,[17]
9,FCLT_BUILDING,"[4, 39, 45, 13, 46]"


# Treat ChatGPT as an all-knowing oracle (without any additional information)
## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at 
MIT, containing information about students, faculty, and personnel. You can find more about the MIT data 
warehouse tables at https://web.mit.edu/warehouse/metadata/tables/all_tables.html._

_How many organisation units in human resources are relabeled?_

In [67]:
api_key = config['api-key']
client = OpenAI(api_key=api_key)

## Populate the prompt and call the Open AI API

In [75]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

response = requests.get('https://api.openai.com/v1/engines', headers=headers)

if response.status_code == 200:
    engines = response.json()['data']
    print("Available engines:")
    for engine in engines:
        print(engine['id'])
else:
    print("Error:", response.text)

Available engines:
dall-e-3
whisper-1
davinci-002
gpt-3.5-turbo-1106
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-4
gpt-4-turbo-2024-04-09
gpt-4-0613
gpt-4-1106-preview
gpt-4-vision-preview
gpt-3.5-turbo
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0613
gpt-3.5-turbo-instruct
gpt-4-1106-vision-preview
gpt-3.5-turbo-0125
tts-1
gpt-3.5-turbo-0301
babbage-002
tts-1-1106
gpt-4-0125-preview
text-embedding-3-large
gpt-4-turbo-preview
gpt-3.5-turbo-16k-0613
text-embedding-3-small
text-embedding-ada-002
gpt-4-turbo


In [ ]:
# TODO: do this in batch 
gpt_sql_nocontext = []
for q in df_q['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages = [
          {
          "role": "system",
          "content": "You are a OracleSQL expert. Given an input question, create a syntactically correct OracleSQL query to run and return ONLY the generated Query and nothing else.\
            Pay close attention on which column is in which table. if context contains more than one tables then create a query by performing JOIN operation only using the column unitid for the tables.\
Follow these Instructions for creating syntactically correct SQL query:\
- Be sure not to query for columns that do not exist in the tables and use alias only where required.\
- Likewise, when asked about the average (AVG function) or ratio, ensure the appropriate aggregation function is used.\
- Pay close attention to the filtering criteria mentioned in the question and incorporate them using the WHERE clause in your SQL query.\
- If the question involves multiple conditions, use logical operators such as AND, OR to combine them effectively.\
- When dealing with date or timestamp columns, use appropriate date functions (e.g., DATE_PART, EXTRACT) for extracting specific parts of the date or performing date arithmetic.\
- If the question involves grouping of data (e.g., finding totals or averages for different categories), use the GROUP BY clause along with appropriate aggregate functions.\
- Consider using aliases for tables and columns to improve readability of the query, especially in case of complex joins or subqueries.\
- If necessary, use subqueries or common table expressions (CTEs) to break down the problem into smaller, more manageable parts.\
Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, buildings, and organisation. You can find more about the MIT data warehouse tables at \
          https://web.mit.edu/warehouse/metadata/tables/all_tables.html. \
          Reply with only the answer in SQL and include no linebreaks, newlines, escape characters or other commentary"
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
          },
            {"role":"user", 
             "content":q}]
    )
    gpt_sql_nocontext.append(response.choices[0].message.content)

In [ ]:
gpt_sql_nocontext

In [ ]:
len(gpt_sql_nocontext)

In [ ]:
df_q['gpt_sql_nocontext']=gpt_sql_nocontext

# Give ChatGPT the "database" schema as context and elicit its answer


In [ ]:
def table_to_sql_create(t): 
    schema_file=str(prefix) + '/data/views/schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE "+ t+ "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
    return p
    

In [ ]:
prompt = "** all the tables in the database ** \n\n"
cnt = 0 
for t in unique_tables:
    if t.upper().startswith('WAREUSER.'):
        t = t[len('WAREUSER.'):]
    p = table_to_sql_create(t)
    if not p: 
        print(t)
    if p: 
        cnt = cnt + 1
        prompt = prompt + p
        prompt = prompt + "\n\n"

In [ ]:
print(len(prompt))
additional_space = 3000-len(prompt)

## Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing 
information about students, faculty, and personnel. The schemas of all the tables that you might need 
for answering the user questions are below. There are 50 tables in the database.  Reply with as many answers 
as you can think of in SQL and include no linebreaks, newlines, escape characters or other commentary for the each possible 
answer, but seperate the possibilities by semicolons. + "alltable_schemas"_

In [ ]:
gpt_sql_global_schema = []
for q in df_q['question']: 
    response = client.chat.completions.create(
        model="gpt-4",
        temperature = 0,
        messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of all the tables that you might need \
          for answering the user questions are below. There are 39 tables in the database. Reply with only the \
          answer in SQL and include no linebreaks, newlines, escape characters or other commentary. \n\n" + prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_global_schema.append(response.choices[0].message.content)
    print(response.choices[0].message.content)

In [ ]:
df_q['gpt_sql_global_schema']

In [ ]:
df_q['gpt_sql_global_schema'] = gpt_sql_global_schema

# Give ChatGPT the "database" schema and prev queries as context and elicit its answer
### Prompt
_Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. 
The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing 
information about students, faculty, and personnel. The schemas of the all the tables that you might need 
for answering the user questions are given below. There are 47 tables in the database. 
For each table, the schema of the table and the previous queries that referred to the table are also given. \
Reply with as many answers as you can think of in SQL and include no linebreaks, newlines, escape characters or other 
commentary for the each possible  answer, but seperate the possibilities by semicolons_

In [54]:
def min_arg(l):
    min_a = list(l)[0]
    for i in l:
        if len(df_q['sql'][i]) < len(df_q['sql'][min_a]):
            min_a = i
    return min_a


In [17]:
def table_to_sql_create_with_logs(t, l, used_querries, cnter, maxqueriespertable, allowduplicates): 
    schema_file= str(prefix) + '/data/views/schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t_ = pd.read_csv(schema_file)
        p = "CREATE TABLE " + t + "(\n" 
        p = p + ',\n'.join(list(df_t_[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
        if l and cnter > 0:
            shortest_query_index = min_arg(l)
            sorted_indexes = sorted(l, key=lambda i: len(df_q['sql'][i]))
            if not all(df_q['sql'][item] in used_querries for item in sorted_indexes) or allowduplicates:
                p = p + "\n ** PREVIOUS SQL QUERIES THAT REFERRED TO THE TABLE {0} **\n".format(t)
                for i in range(len(sorted_indexes)):
                    shortest_query = df_q['sql'][sorted_indexes[i]]
                    if (shortest_query not in used_querries or allowduplicates) and maxqueriespertable>0:
                        p = p + "SQL: {0} \n".format(shortest_query)
                        used_querries.append(shortest_query)
                        cnter -= 1
                        maxqueriespertable-=1
    return p, cnter

In [298]:
def create_prompt(curr_query_index): 
    prompt = "*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** \n"
    cnt = 50 # number of querries
    used_querries=[]
    for i, t in enumerate(unique_tables):
        l = set(df_t['log'][i]).difference([curr_query_index])
        p,cnt = table_to_sql_create_with_logs(t, l, used_querries, cnt, 50, False)
        if not p:
            print(t)
        else: 
            prompt = prompt + p
            prompt = prompt + "\n"
    return prompt 

In [299]:
 print(len(create_prompt(0)))

33369


In [300]:
gpt_sql_log = []
for i, q in enumerate(df_q['question']): 
    prompt = create_prompt(i)
    print(len(prompt))
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      temperature = 0,
      messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of the all the tables that you might need \
          for answering the user questions are given below. There are 39 tables in the database. \
          For each table, the schema of the table and the previous queries that referred to the table are also given. \
          \
          Reply with only the answer in SQL and include no linebreaks, newlines, escape characters or other commentary \n\n" + prompt 
          # prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    print(response.choices[0].message.content)
    gpt_sql_log.append(response.choices[0].message.content)

33369
SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_CODE_OLD IS NOT NULL;
33373
SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_LEVEL = 'New' AND HR_DEPARTMENT_CODE_OLD IS NULL;
33313
SELECT department_name, COUNT(*) FROM COURSE_CATALOG_SUBJECT_OFFERED GROUP BY department_name;
33175
SELECT DISTINCT SUBJECT_TITLE FROM COURSE_CATALOG_SUBJECT_OFFERED WHERE TERM_CODE < (SELECT MIN(TERM_CODE) FROM ACADEMIC_TERMS_ALL) OR TERM_CODE > (SELECT MAX(TERM_CODE) FROM ACADEMIC_TERMS_ALL) OR TERM_CODE = (SELECT MAX(TERM_CODE) FROM ACADEMIC_TERMS_ALL);
33289
SELECT department_name, COUNT(DISTINCT office_location) AS num_location_changes FROM employee_directory GROUP BY department_name ORDER BY num_location_changes DESC FETCH FIRST ROW ONLY;
33342
SELECT AVG(SUBJECT_ENROLLMENT_NUMBER) FROM SUBJECT_SUMMARY;
33362
SELECT DISTINCT full_name FROM mit_student_directory WHERE mit_id IN (SELECT mit_id FROM employee_directory);
33193
SELECT MIN(TOTAL_UNITS) FROM SUBJECT_SUMMARY;
33261
SELECT BUI

In [301]:
df_q['gpt_sql_log_50_all'] = gpt_sql_log

In [191]:
df_q.keys()

Index(['question', 'sql', 'tables', 'gpt_sql_log_50_all', 'gpt_sql_log_50_1',
       'gpt_sql_log_25_1', 'gpt_sql_log_10_1', 'gpt_sql',
       'gpt_sql_logs_50_all'],
      dtype='object')

# Give ChatGPT the "database" schema and prev queries as context as well as their corresponding questions

In [55]:
def table_to_sql_create_with_logs_question_pairs(t, l, used_querries, cnter, maxqueriespertable, allowduplicates): 
    schema_file= str(prefix) + '/data/views/schema_new/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t_ = pd.read_csv(schema_file)
        p = "CREATE TABLE " + t + "(\n" 
        p = p + ',\n'.join(list(df_t_[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
        if l and cnter > 0:
            sorted_indexes = sorted(l, key=lambda i: len(df_q['sql'][i]))
            if not all(df_q['sql'][item] in used_querries for item in sorted_indexes) or allowduplicates:
                p = p + "\n ** PREVIOUS QUESTIONS AND CORRESPONDING SQL QUERIES THAT REFERRED TO THE TABLE {0} **\n".format(t)
                for i in range(len(sorted_indexes)):
                    shortest_query = df_q['sql'][sorted_indexes[i]]
                    if (shortest_query not in used_querries or allowduplicates) and maxqueriespertable>0:
                        p = p + "Question: {0} \n".format(df_q['question'][sorted_indexes[i]])
                        p = p + "SQL: {0} \n".format(shortest_query)
                        used_querries.append(shortest_query)
                        cnter -= 1
                        maxqueriespertable-=1
    return p, cnter

In [56]:
def create_prompt_for_q_a_pairs(curr_query_index, num_of_queries): 
    prompt = "*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** \n"
    cnt = num_of_queries # number of querries
    used_querries=[]
    for i, t in enumerate(unique_tables):
        l = set(df_t['log'][i]).difference([curr_query_index])
        p,cnt = table_to_sql_create_with_logs_question_pairs(t, l, used_querries, cnt, 50, True)
        if not p:
            print(t)
        else: 
            prompt = prompt + p
            prompt = prompt + "\n"
    return prompt 

In [ ]:
def create_prompt_for_q_a_pairs(curr_query_index, num_of_queries): 
    prompt = "*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** \n"
    cnt = num_of_queries # number of querries
    used_querries=[]
    for i, t in enumerate(unique_tables):
        l = set(df_t['log'][i]).difference([curr_query_index])
        p,cnt = table_to_sql_create_with_logs_question_pairs(t, l, used_querries, cnt, 50, True)
        if not p:
            print(t)
        else: 
            prompt = prompt + p
            prompt = prompt + "\n"
    return prompt 

In [65]:
print(len(create_prompt_for_q_a_pairs(0, 100)))

57335


In [68]:
for h in range(100,110,10):
    for j in range(3):
        gpt_sql_log = []
        for i, q in enumerate(df_q['question']): 
            prompt = create_prompt_for_q_a_pairs(i, h)
            print(len(prompt))
            response = client.chat.completions.create(
            model="gpt-3.5-turbo",
              temperature = 0,
              top_p=1.0,
              seed=42,
              messages = [
                {
                "role": "system",
                "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
                The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
                information about students, faculty, and personnel. The schemas of the all the tables that you might need \
                for answering the user questions are given below. There are 39 tables in the database. \
                For each table, the schema of the table and the previous queries that referred to the table are also given. \
                \
                Reply with only the answer in SQL and include no linebreaks, newlines, escape characters or other commentary \n\n" + prompt 
              # prompt 
              # Provide your answer in JSON form."
              # Reply with only the answer in JSON form and include no other commentary"
                },
                {"role":"user", 
               "content":q}]
                )
            print(response.choices[0].message.content)
            gpt_sql_log.append(response.choices[0].message.content)
        df_q["gpt_sql_q_a_log_{0}_all.{1}".format(str(h), str(j))] = gpt_sql_log


57335
SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit MINUS SELECT hr_org_unit_key FROM hr_org_unit_new);
57319
SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new EXCEPT SELECT hr_org_unit_key FROM hr_org_unit);
57454
SELECT DEPARTMENT_NAME, COUNT(*) FROM SUBJECT_SUMMARY GROUP BY DEPARTMENT_NAME;
56971
SELECT DISTINCT subject_title FROM SUBJECT_SUMMARY WHERE term_start_date < SYSDATE OR term_start_date > SYSDATE OR term_start_date = SYSDATE;
57256
SELECT * FROM (SELECT FULL_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS num_locations FROM EMPLOYEE_DIRECTORY GROUP BY FULL_NAME ORDER BY num_locations DESC) WHERE ROWNUM = 1;
57279
SELECT AVG(SUBJECT_ENROLLMENT_NUMBER) FROM SUBJECT_SUMMARY;
57373
SELECT DISTINCT full_name 
FROM mit_student_directory 
WHERE full_name IN (SELECT full_name 
                    FROM employee_directory);
57001
SELECT SUBJECT_TITLE FROM SUBJECT_SUMMARY WHERE TOTAL_UNITS = (SELECT MIN(TOTAL_UNITS) FROM SUBJECT_SUMMARY);
57329
SELECT BUILDIN

In [59]:
for i in range(0,60,10):
    sql=[]
    for j in range(len(df_q)):
        if df_q["gpt_sql_q_a_log_{0}_1.0".format(str(i))][j] == df_q["gpt_sql_q_a_log_{0}_1.1".format(str(i))][j]:
            sql.append(df_q["gpt_sql_q_a_log_{0}_1.0".format(str(i))][j])
        elif df_q["gpt_sql_q_a_log_{0}_1.1".format(str(i))][j] == df_q["gpt_sql_q_a_log_{0}_1.2".format(str(i))][j]:
            sql.append(df_q["gpt_sql_q_a_log_{0}_1.1".format(str(i))][j])
        elif df_q["gpt_sql_q_a_log_{0}_1.0".format(str(i))][j] == df_q["gpt_sql_q_a_log_{0}_1.2".format(str(i))][j]:
            sql.append(df_q["gpt_sql_q_a_log_{0}_1.2".format(str(i))][j])
        else:
            print("oh wow: " + str(j) +" + " + str(i)) 
            sql.append(df_q["gpt_sql_q_a_log_{0}_1.2".format(str(i))][j])
    df_q["gpt_sql_q_a_log_{0}_1".format(str(i))] = sql
    print(i)

oh wow: 1 + 0
oh wow: 4 + 0
oh wow: 6 + 0
oh wow: 30 + 0
oh wow: 39 + 0
oh wow: 47 + 0
0
oh wow: 1 + 10
oh wow: 7 + 10
oh wow: 8 + 10
oh wow: 16 + 10
oh wow: 36 + 10
oh wow: 48 + 10
10
oh wow: 16 + 20
oh wow: 24 + 20
oh wow: 33 + 20
oh wow: 39 + 20
oh wow: 42 + 20
oh wow: 44 + 20
20
oh wow: 3 + 30
oh wow: 6 + 30
oh wow: 27 + 30
oh wow: 37 + 30
oh wow: 39 + 30
30
oh wow: 18 + 40
oh wow: 32 + 40
oh wow: 39 + 40
40
oh wow: 3 + 50
oh wow: 42 + 50
oh wow: 46 + 50
50


In [107]:
df_q[['gpt_4_sql_q_a_log_20_1.0', 'gpt_4_sql_q_a_log_20_1.1', 'gpt_4_sql_q_a_log_20_1.2']]

,gpt_4_sql_q_a_log_20_1.0,gpt_4_sql_q_a_log_20_1.1,gpt_4_sql_q_a_log_20_1.2
0,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit INTERSECT SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit INTERSECT SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit INTERSECT SELECT hr_org_unit_key FROM hr_org_unit_new);
1,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);
2,"SELECT OFFER_DEPT_NAME, COUNT(DISTINCT MASTER_COURSE_NUMBER) FROM LIBRARY_SUBJECT_OFFERED GROUP BY OFFER_DEPT_NAME;","SELECT OFFER_DEPT_NAME, COUNT(DISTINCT MASTER_COURSE_NUMBER) FROM LIBRARY_SUBJECT_OFFERED GROUP BY OFFER_DEPT_NAME;","SELECT OFFER_DEPT_NAME, COUNT(DISTINCT MASTER_COURSE_NUMBER) FROM LIBRARY_SUBJECT_OFFERED GROUP BY OFFER_DEPT_NAME;"
3,SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code < (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code = (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code > (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y');,SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code < (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code = (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code > (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y');,SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code < (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code = (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y') UNION SELECT DISTINCT subject_title FROM course_catalog_subject_offered WHERE term_code > (SELECT term_code FROM academic_terms_all WHERE is_current_term = 'Y');
4,"SELECT * FROM (SELECT RESPONSIBLE_FACULTY_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS num_location_changes FROM LIBRARY_SUBJECT_OFFERED GROUP BY RESPONSIBLE_FACULTY_NAME ORDER BY num_location_changes DESC) WHERE ROWNUM <= 1;","SELECT * FROM (SELECT RESPONSIBLE_FACULTY_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS num_location_changes FROM SE_PERSON GROUP BY RESPONSIBLE_FACULTY_NAME ORDER BY num_location_changes DESC) WHERE ROWNUM <= 1;","SELECT * FROM (SELECT responsible_faculty_name, COUNT(DISTINCT office_location) AS num_location_changes FROM se_person GROUP BY responsible_faculty_name ORDER BY num_location_changes DESC) WHERE ROWNUM <= 1;"
5,SELECT AVG(SUBJECT_ENROLLMENT_NUMBER) FROM SUBJECT_SUMMARY;,SELECT AVG(SUBJECT_ENROLLMENT_NUMBER) FROM SUBJECT_SUMMARY;,SELECT AVG(SUBJECT_ENROLLMENT_NUMBER) FROM SUBJECT_SUMMARY;
6,SELECT DISTINCT MIT_ID FROM MIT_STUDENT_DIRECTORY;,"SELECT DISTINCT mit_id, full_name FROM mit_student_directory WHERE mit_id IN (SELECT mit_id FROM se_person WHERE employee_type = 'Student');","SELECT DISTINCT mit_id, full_name FROM mit_student_directory WHERE mit_id IN (SELECT mit_id FROM se_person WHERE employee_type = 'Student');"
7,SELECT MIN(TOTAL_UNITS) FROM COURSE_CATALOG_SUBJECT_OFFERED;,SELECT MIN(TOTAL_UNITS) FROM COURSE_CATALOG_SUBJECT_OFFERED;,SELECT MIN(TOTAL_UNITS) FROM COURSE_CATALOG_SUBJECT_OFFERED;
8,"SELECT BUILDING_NAME_LONG, STREET_NUMBER, STREET_NAME, CITY, STATE, POSTAL_CODE FROM FAC_BUI

In [17]:
for j in range(0,60,10):
    sql = []
    for i in range(len(df_q)):
        if df_q['gpt_sql_q_a_log_{0}_1.0'.format(j)][i] == df_q['gpt_sql_q_a_log_{0}_1.1'.format(j)][i]:
            sql.append(df_q['gpt_sql_q_a_log_{0}_1.0'.format(j)][i])
        elif df_q['gpt_sql_q_a_log_{0}_1.1'.format(j)][i] == df_q['gpt_sql_q_a_log_{0}_1.2'.format(j)][i]:
            sql.append(df_q['gpt_sql_q_a_log_{0}_1.1'.format(j)][i])
        else:
            sql.append(df_q['gpt_sql_q_a_log_{0}_1.2'.format(j)][i])
    df_q['gpt_sql_q_a_log_{0}_1'.format(j)]=sql

In [109]:
df_q['gpt_sql_q_a_log_20_1.0'] = sql

# Give ChatGPT the "database" schema and prev queries as context as well as their corresponding questions

# READ DATAFRAME


In [15]:
df_q = pd.read_csv(str(prefix) + '/data/gpt_eval_50querries_noAmb_v1.0.csv')

In [16]:
df_q

,question,sql,tables,gpt_sql_q_a_log_20_1,gpt_sql_q_a_log_20_1.0,gpt_sql_q_a_log_20_1.1,gpt_sql_q_a_log_20_1.2,gpt_sql_q_a_log_0_1.0,gpt_sql_q_a_log_0_1.1,gpt_sql_q_a_log_0_1.2,gpt_sql_q_a_log_10_1.0,gpt_sql_q_a_log_10_1.1,gpt_sql_q_a_log_10_1.2,gpt_sql_q_a_log_30_1.0,gpt_sql_q_a_log_30_1.1,gpt_sql_q_a_log_30_1.2,gpt_sql_q_a_log_40_1.0,gpt_sql_q_a_log_40_1.1,gpt_sql_q_a_log_40_1.2,gpt_sql_q_a_log_50_1.0,gpt_sql_q_a_log_50_1.1,gpt_sql_q_a_log_50_1.2
0,How many organisation units in human resources are relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);,"['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']",SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit INTERSECT SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(DISTINCT HR_ORG_UNIT_KEY) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG,SELECT COUNT(DISTINCT HR_ORG_UNIT_KEY) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG,SELECT COUNT(DISTINCT HR_ORG_UNIT_KEY) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_ALPHA;,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_ALPHA;,SELECT COUNT(*) FROM HR_ORG_UNIT WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_ALPHA;,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit MINUS SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);,SELECT COUNT(*) FROM hr_org_unit WHERE hr_org_unit_key IN (SELECT hr_org_unit_key FROM hr_org_unit_new);
1,How many organisation units in human resources are new and not relabeled?,SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);,"['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']",SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*) FROM hr_org_unit_new WHERE hr_org_unit_key NOT IN (SELECT hr_org_unit_key FROM hr_org_unit);,SELECT COUNT(*)\nFROM HR_ORG_UNIT_NEW\nWHERE WAREHOUSE_LOAD_DATE = (SELECT MAX(WAREHOUSE_LOAD_DATE) FROM HR_ORG_UNIT_NEW)\nAND HR_ORG_UNIT_LEVEL = 'NEW'\nAND HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG,SELECT COUNT(*) \nFROM HR_ORG_UNIT_NEW \nWHERE HR_DEPARTMENT_CODE_OLD IS NULL,SELECT COUNT(*)\nFROM HR_ORG_UNIT_NEW\nWHERE HR_ORG_UNIT_LEVEL = 'NEW' AND HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_NAME NOT LIKE '%relabeled%';,SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_CODE_OLD IS NULL;,SELECT COUNT(HR_ORG_UNIT_KEY) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_LEVEL = 'NEW' AND HR_DEPARTMENT_NAME != HR_DEPAR

In [ ]:
new_row = {'question':"What kind of questions can I ask the data warehoues?", 'sql':["SELECT DISTINCT ST.STAR_NAME, REPLACE (REPLACE (ST.DESCRIPTION, CHR(13)), CHR(10), ' ') AS STAR_DESCRIPTION,  UV.VIEW_NAME, REPLACE (REPLACE (UV.DESCRIPTION, CHR(13)), CHR(10), ' ') AS VIEW_DESCRIPTION, F.FIELD_NAME, F.DEFINITION AS FIELD_DESCRIPTION  FROM STAR ST JOIN STAR_USER_VIEW SAV ON ST.STAR_ID = SAV.STAR_ID JOIN USER_VIEW UV ON SAV.VIEW_ID = UV.VIEW_ID JOIN FIELDS F ON F.TABLE_NAME = UV.VIEW_NAME ORDER BY 1, 3, 4;"],
       'tables':[], 'gpt_sql_nocontext':[],
       'gpt_sql_global_schema':[], 'expected_responses':[],
       'responses_gpt_nocontent':[], 'failures_gpt_nocontent':[],
       'accuracies_nocontext':[], 'precisions_nocontext':[], 'recalls_nocontext':[],
       'f1_scores_nocontext':[], 'responses_gpt_global_schema':[],
       'failures_gpt_global_schema':[], 'accuracies_global_schema':[],
       'precisions_global_schema':[], 'recalls_global_schema':[],
       'f1_scores_global_schema':[]}
new_df = pd.DataFrame([new_row])
df_q = pd.concat([df_q, new_df], ignore_index=True)

In [ ]:
sql = []
tables = []
gpt_sql_nocontext = []
gpt_sql_global_schema = []
expected_responses = []
for i in range(len(df_q)):
    sql.append(ast.literal_eval(df_q['sql'][i]))
    tables.append(ast.literal_eval(df_q['tables'][i]))
    gpt_sql_nocontext.append(ast.literal_eval(df_q['gpt_sql_nocontext'][i]))
    gpt_sql_global_schema.append(ast.literal_eval(df_q['gpt_sql_global_schema'][i]))
    #expected_responses.append(ast.literal_eval(df_q['expected_responses'][i]))
    #responses_gpt_nocontent.append(ast.literal_eval(df_q['responses_gpt_nocontent'][i]))
    #responses_gpt_global_schema.append(ast.literal_eval(df_q['responses_gpt_global_schema'][i]))
    #tables_referred_by_gpt_sql_global_schema.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_global_schema'][i]))
    #tables_referred_by_gpt_sql_nocontext.append(ast.literal_eval(df_q['tables_referred_by_gpt_sql_nocontext'][i]))
    #sql_outputsizes.append(ast.literal_eval(df_q['sql_outputsizes'][i]))
df_q['sql'] = sql
df_q['tables'] = tables
df_q['gpt_sql_global_schema'] = gpt_sql_global_schema
df_q['gpt_sql_nocontext'] = gpt_sql_nocontext
#df_q['sql_outputsizes'] = sql_outputsizes

In [ ]:
sql = []
tables = []
gpt_sql_global_schema = []
gpt_sql_nocontext = []
question = []
gpt_global_schema = []
for i in range(len(df_q)):
    if i == 0 or df_q['question'][i] != df_q['question'][i-1]:
        if gpt_global_schema != None and i != 0:
            gpt_sql_global_schema.append(gpt_global_schema)
        gpt_global_schema = []
        sql.append(df_q['sql'][i])
        tables.append(df_q['tables'][i])
        gpt_global_schema.append(df_q['gpt_sql_global_schema'][i])
        gpt_sql_nocontext.append(df_q['gpt_sql_nocontext'][i])
        question.append(df_q['question'][i])
    else:
        gpt_global_schema.append(df_q['gpt_sql_global_schema'][i])
gpt_sql_global_schema.append(gpt_global_schema)
df_temp = pd.DataFrame(data={"question":question, "sql":sql, "tables":tables, "gpt_sql_global_schema":gpt_sql_global_schema, "gpt_sql_nocontext":gpt_sql_nocontext})

In [ ]:
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df_q

# Generated for all answers the actual output

In [21]:

def execute_query(query, tables):
  connection = cx_Oracle.connect(user=config['username'], password=config['password'], dsn=config['dsn'])

  if 'LIMIT' in query:
    contains_limit = True
    limit_number = re.search(r'\bLIMIT\s+(\d+)', query)
    limit_value = int(limit_number.group(1))
    query = query[:query.index('LIMIT ')]
    query = 'SELECT * FROM (' + query + ') WHERE ROWNUM <=' + str(limit_value)

  # Set input type handler to cx_Oracle.STRING
  connection.inputtypehandler = cx_Oracle.STRING
  # Create a cursor object
  cursor = connection.cursor()

  # Set DEFINE OFF
  cursor.setoutputsize(0)

  # Execute SQL queries
  try:
    cursor.execute(query)
    # Fetch the results
    rows = cursor.fetchall()
  except Exception as e1:
    # Preprocessing wueries
    for table in tables:
      table_mod = ' WAREUSER.' + table
      query = query.replace(' ' + table, table_mod)
      print(query)
    try:
      cursor.execute(query)
      # Fetch the results
      rows = cursor.fetchall()
    except Exception as e2:
      # Print the error message to stderr
      print("An error occurred:", str(e1))
      print("An error occurred:", str(e2))
      return None
  cursor.close()
  connection.close()
  return rows

## Start with the correct answers

In [22]:

gold_answers = []
failures = []
for q in df_q['sql']:
    print(q)
    try:
        gold_answers.append(execute_query(q[:-1], [v.upper() for v in Parser(q).tables]))
        failures.append(None)
    except Exception as e:
        gold_answers.append(None)
        failures.append(str(e))
    print(failures)
    print('------')

SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new INTERSECT SELECT hr_org_unit_key FROM hr_org_unit);
[None]
------
SELECT COUNT(*) FROM (SELECT hr_org_unit_key FROM hr_org_unit_new MINUS SELECT hr_org_unit_key FROM hr_org_unit);
[None, None]
------
SELECT OFFER_DEPT_CODE, COUNT(*) AS course_COUNT FROM SUBJECT_ENROLLABLE GROUP BY OFFER_DEPT_CODE;
[None, None, None]
------
SELECT DISTINCT subject_title, academic_terms.term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM SUBJECT_SUMMARY, academic_terms WHERE SUBJECT_SUMMARY.term_code = academic_terms.term_code;
[None, None, None, None]
------
SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt FROM WAREUSER.FCLT_BUILDING_ADDRESS_HIST GROUP BY fclt_building_key);
[None, None, None, None, None]
------
SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt FROM mit_student_directory, SUBJECT_SUMMARY WHERE DEPARTMENT_CODE =

In [23]:
df_q['expected_responses'] = gold_answers
if not all(f is None for f in failures):
    print("Please check errors!")

## Continue with the answers from gpt with no additional information

In [ ]:
gold_answers = []
failures = []
for q in df_q['gpt_sql_nocontext']:
    try:
        if q[len(q)-1]==';':
            q = q[:-1]
        print(q)
        gold_answers.append(execute_query(q, [v.upper() for v in Parser(q).tables]))
        failures.append(None)
    except Exception as e:
        gold_answers.append(None)
        failures.append(str(e))

In [ ]:
df_q['responses_gpt_nocontent'] = gold_answers
df_q['failures_gpt_nocontent'] = failures

## Continue with the answers from gpt with schema information

In [18]:
df_q.keys()

Index(['question', 'sql', 'tables', 'gpt_sql_q_a_log_20_1',
       'gpt_sql_q_a_log_20_1.0', 'gpt_sql_q_a_log_20_1.1',
       'gpt_sql_q_a_log_20_1.2', 'gpt_sql_q_a_log_0_1.0',
       'gpt_sql_q_a_log_0_1.1', 'gpt_sql_q_a_log_0_1.2',
       'gpt_sql_q_a_log_10_1.0', 'gpt_sql_q_a_log_10_1.1',
       'gpt_sql_q_a_log_10_1.2', 'gpt_sql_q_a_log_30_1.0',
       'gpt_sql_q_a_log_30_1.1', 'gpt_sql_q_a_log_30_1.2',
       'gpt_sql_q_a_log_40_1.0', 'gpt_sql_q_a_log_40_1.1',
       'gpt_sql_q_a_log_40_1.2', 'gpt_sql_q_a_log_50_1.0',
       'gpt_sql_q_a_log_50_1.1', 'gpt_sql_q_a_log_50_1.2',
       'gpt_sql_q_a_log_0_1', 'gpt_sql_q_a_log_10_1', 'gpt_sql_q_a_log_30_1',
       'gpt_sql_q_a_log_40_1', 'gpt_sql_q_a_log_50_1'],
      dtype='object')

In [ ]:
gold_answers = []
failures = []
for q in df_q['gpt_sql_global_schema']:
    try:
        if q[len(q)-1]==';':
            q = q[:-1]
        print(q)
        gold_answers.append(execute_query(q, [v.upper() for v in Parser(q).tables]))
        failures.append(None)
    except Exception as e:
        gold_answers.append(None)
        failures.append(str(e))
    print(failures)
    print('------')

In [ ]:
df_q['responses_gpt_global_schema'] = gold_answers
df_q['failures_gpt_global_schema'] = failures

## Continue with the answers from gpt with schema information and query logs

In [24]:
for j in range(0,60,10):
    gold_answers = []
    failures = []
    for q in df_q['gpt_sql_q_a_log_{0}_1'.format(j)]:
        try:
            if q[len(q)-1]==';':
                q = q[:-1]
            print(q)
            gold_answers.append(execute_query(q, [v.upper() for v in Parser(q).tables]))
            failures.append(None)
        except Exception as e:
            gold_answers.append(None)
            failures.append(str(e))
        print(failures)
        print('------')
    df_q['responses_gpt_sql_q_a_log_{0}_1'.format(j)] = gold_answers

SELECT COUNT(DISTINCT HR_ORG_UNIT_KEY) FROM HR_ORG_UNIT_NEW WHERE HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG
[None]
------
SELECT COUNT(*)
FROM HR_ORG_UNIT_NEW
WHERE HR_ORG_UNIT_LEVEL = 'NEW' AND HR_DEPARTMENT_NAME <> HR_DEPARTMENT_NAME_LONG
[None, None]
------
SELECT DEPARTMENT_CODE, COUNT(DISTINCT SUBJECT_ID) AS NUM_COURSES
FROM COURSE_CATALOG_SUBJECT_OFFERED
GROUP BY DEPARTMENT_CODE
[None, None, None]
------
SELECT DISTINCT SUBJECT_TITLE
FROM SUBJECT_OFFERED
WHERE TERM_CODE < '2023'
UNION
SELECT DISTINCT SUBJECT_TITLE
FROM SUBJECT_OFFERED
WHERE TERM_CODE = '2023'
UNION
SELECT DISTINCT SUBJECT_TITLE
FROM SUBJECT_OFFERED
WHERE TERM_CODE > '2023'
[None, None, None, None]
------
SELECT ORGANIZATION, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_COUNT
FROM SE_PERSON
GROUP BY ORGANIZATION
ORDER BY LOCATION_COUNT DESC
FETCH FIRST ROW ONLY
SELECT ORGANIZATION, COUNT(DISTINCT OFFICE_LOCATION) AS LOCATION_COUNT
FROM WAREUSER.SE_PERSON
GROUP BY ORGANIZATION
ORDER BY LOCATION_COUNT DESC
FETCH 

In [66]:
df_q['responses_gpt_sql_q_a_log_20_1'] = gold_answers
#df_q['failures_gpt_sql_log'] = failures

# Calculate Accuracy, Precision, Recall and F1-Score

Accuracy: $\frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$

Precision: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}$

Recall: $\frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}$

F1-Score: $2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

In [26]:
def metric_(gold_set, prediction_set, recall=None):
  if gold_set is None or prediction_set is None:
    return np.array([0, 0, 0])
  if len(gold_set) == len(prediction_set) == 0:
    return np.array([1, 1, 1])

  true_positives = len(gold_set.intersection(prediction_set))
  false_positives = len(prediction_set - gold_set)
  false_negatives = len(gold_set - prediction_set)

  assert true_positives + false_positives == len(prediction_set)
  assert true_positives + false_negatives == len(gold_set)

  precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 1

  if recall is None:
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
  f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
  return np.array([precision, recall, f1])

In [32]:
a, b, c, d = [], [], [], []
for j in range(0,60,10):
    accuracies_nocontext, precisions_nocontext, recalls_nocontext, f1_scores_nocontext = calculate_metrics_for('responses_gpt_sql_q_a_log_{0}_1'.format(j))
    print(j)
    a.append(sum(accuracies_nocontext)/(len(df_q)))
    b.append(sum(precisions_nocontext)/(len(df_q)))
    c.append(sum(recalls_nocontext)/(len(df_q)))
    d.append(sum(f1_scores_nocontext)/(len(df_q)))
    print(sum(accuracies_nocontext)/(len(df_q)))
    print(sum(precisions_nocontext)/(len(df_q)))
    print(sum(recalls_nocontext)/(len(df_q)))
    print(sum(f1_scores_nocontext)/(len(df_q)))

0
0.16
0.2969417475728155
0.19384615384615386
0.19470792020069286
10
0.12
0.2956660214014537
0.1920650883365159
0.1931762750870442
20
0.14
0.31567014863354703
0.2403731396784062
0.23360394578240598
30
0.12
0.3001670308667636
0.217013177136708
0.19461386463200966
40
0.12
0.26006777528363706
0.21885391286082004
0.19556096792586444
50
0.12
0.26006777528363706
0.21885391286082004
0.19556096792586444


In [36]:
d

[0.19470792020069286,
 0.1931762750870442,
 0.23360394578240598,
 0.19461386463200966,
 0.19556096792586444,
 0.19556096792586444]

In [28]:
def calculate_metrics_for(gpt_format:str):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    if 'expected_responses' in df_q.keys():
        for i in range(len(df_q)):
            if df_q['expected_responses'][i] is None or df_q[gpt_format][i] is None:
                accuracy_val,precision_val, recall_val, f1_val=  0,0,0,0
            else:
                gold = set(df_q['expected_responses'][i])
                prediction = set(df_q[gpt_format][i])
                accuracy_val = int(gold == prediction)
                precision_val, recall_val, f1_val = metric_(gold, prediction)
            accuracies.append(accuracy_val)
            precisions.append(precision_val)
            recalls.append(recall_val)
            f1_scores.append(f1_val)
    return [accuracies, precisions, recalls, f1_scores]


In [ ]:
accuracies_nocontext, precisions_nocontext, recalls_nocontext, f1_scores_nocontext = [], [], [], []
accuracies_nocontext, precisions_nocontext, recalls_nocontext, f1_scores_nocontext = calculate_metrics_for('responses_gpt_nocontent')
df_q['accuracies_nocontext'] = accuracies_nocontext
df_q['precisions_nocontext'] = precisions_nocontext
df_q['recalls_nocontext'] = recalls_nocontext
df_q['f1_scores_nocontext'] = f1_scores_nocontext

In [ ]:
arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
for metric in arrray_of_metrics:
    print(metric + ' for nocontext: ' + str(sum(df_q[metric + '_nocontext'])/(len(df_q))))

In [ ]:
accuracies_global_schema, precisions_global_schema, recalls_global_schema, f1_scores_global_schema = calculate_metrics_for('responses_gpt_global_schema')
df_q['accuracies_global_schema'] = accuracies_global_schema
df_q['precisions_global_schema'] = precisions_global_schema
df_q['recalls_global_schema'] = recalls_global_schema
df_q['f1_scores_global_schema'] = f1_scores_global_schema

In [ ]:

for metric in arrray_of_metrics:
    print(metric + ' for global schema: ' + str(sum(df_q[metric + '_global_schema'])/(len(df_q))))

In [ ]:
accuracies_global_schema_log, precisions_global_schema_log, recalls_global_schema_log, f1_scores_global_schema_log = calculate_metrics_for('responses_gpt_sql_log')
df_q['accuracies_global_schema_logs'] = accuracies_global_schema_log
df_q['precisions_global_schema_logs'] = precisions_global_schema_log
df_q['recalls_global_schema_logs'] = recalls_global_schema_log
df_q['f1_scores_global_schema_logs'] = f1_scores_global_schema_log

In [ ]:

for metric in arrray_of_metrics:
    print(metric + ' for global schema & logs: ' + str(sum(df_q[metric + '_global_schema_logs'])/(len(df_q))))

## old accuracies

In [ ]:
arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
for metric in arrray_of_metrics:
    print(metric + ' for nocontext: ' + str(sum([sum(a) for a in df_q[metric + '_nocontext']])/(len(df_q))))

In [ ]:
for metric in arrray_of_metrics:
    print(metric + ' for global schema: ' + str(sum([sum(a) for a in df_q[metric + '_global_schema']])/(len(df_q))))

In [ ]:
arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
for metric in arrray_of_metrics:
    print(metric + ' for nocontext: ' + str(sum([sum(a) for a in df_q[metric + '_nocontext']])/(len(df_q))))

In [ ]:
for metric in arrray_of_metrics:
    print(metric + ' for global schema: ' + str(sum([sum(a) for a in df_q[metric + '_global_schema']])/(len(df_q))))

# Save Dataframe with generated answers


In [57]:
df_q.keys()

Index(['question', 'sql', 'tables', 'gpt_sql_q_a_log_20_1',
       'expected_responses', 'responses_gpt_sql_q_a_log_20_1',
       'gpt_sql_q_a_log_20_1.0', 'gpt_sql_q_a_log_20_1.1',
       'gpt_sql_q_a_log_20_1.2', 'responses_gpt_sql_q_a_log_20_1.0',
       'responses_gpt_sql_q_a_log_20_1.1', 'responses_gpt_sql_q_a_log_20_1.2',
       'gpt_sql_q_a_log_0_1.0', 'gpt_sql_q_a_log_0_1.1',
       'gpt_sql_q_a_log_0_1.2', 'gpt_sql_q_a_log_10_1.0',
       'gpt_sql_q_a_log_10_1.1', 'gpt_sql_q_a_log_10_1.2',
       'gpt_sql_q_a_log_30_1.0', 'gpt_sql_q_a_log_30_1.1',
       'gpt_sql_q_a_log_30_1.2', 'gpt_sql_q_a_log_40_1.0',
       'gpt_sql_q_a_log_40_1.1', 'gpt_sql_q_a_log_40_1.2',
       'gpt_sql_q_a_log_50_1.0', 'gpt_sql_q_a_log_50_1.1',
       'gpt_sql_q_a_log_50_1.2'],
      dtype='object')

In [58]:
df_q[['question', 'sql', 'tables', 'gpt_sql_q_a_log_20_1',
       'gpt_sql_q_a_log_20_1.0', 'gpt_sql_q_a_log_20_1.1',
       'gpt_sql_q_a_log_20_1.2', 'gpt_sql_q_a_log_0_1.0', 'gpt_sql_q_a_log_0_1.1',
       'gpt_sql_q_a_log_0_1.2', 'gpt_sql_q_a_log_10_1.0',
       'gpt_sql_q_a_log_10_1.1', 'gpt_sql_q_a_log_10_1.2',
       'gpt_sql_q_a_log_30_1.0', 'gpt_sql_q_a_log_30_1.1',
       'gpt_sql_q_a_log_30_1.2', 'gpt_sql_q_a_log_40_1.0',
       'gpt_sql_q_a_log_40_1.1', 'gpt_sql_q_a_log_40_1.2',
       'gpt_sql_q_a_log_50_1.0', 'gpt_sql_q_a_log_50_1.1',
       'gpt_sql_q_a_log_50_1.2']].to_csv(str(prefix) + '/data/gpt_eval_50querries_noAmb_v1.0.csv', index=False)

In [37]:
df_q.to_csv(str(prefix) + '/data/gpt_eval_50querries_noAmb_with_resp_v1.0.csv', index=False)

## Save information in file

In [ ]:
def save_accuracies(variant:str):
    current_datetime = datetime.datetime.now()
    file_content = '-->' + variant + '-' + str(current_datetime)+ ':\n'
    arrray_of_metrics = ['accuracies', 'precisions', 'recalls', 'f1_scores']
    for metric in arrray_of_metrics:
        file_content += metric + ' for nocontext: ' + str(sum(df_q[metric + '_nocontext'])/(len(df_q))) + '\n'
    for metric in arrray_of_metrics:
        file_content += metric + ' for global schema: ' + str(sum(df_q[metric + '_global_schema'])/(len(df_q)))+ '\n'
    for metric in arrray_of_metrics:
        file_content += metric + ' for global schema & logs: ' + str(sum(df_q[metric + '_global_schema_logs'])/(len(df_q)))+ '\n'
    with open(str(prefix) + '/data/evaluation.txt', "a") as file:
        file.write(file_content)

In [ ]:
save_accuracies('50_querries_no_ambiguity')